In [1]:
import pandas as pd
import glob

### Import and concatenate all PM10 csv files into one

In [2]:
# Path to the directory containing CSV files 
csv_files_path = r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Data\PM10\*.csv" # Adjust this path as necessary 

# Use glob to get all CSV file paths 
csv_files = glob.glob(csv_files_path) 

# Use list comprehension to read all CSV files and concatenate them into a single DataFrame 
PM10_df = pd.concat([pd.read_csv(file,low_memory=False) for file in csv_files], ignore_index=True) 

# Now PM10_df contains all the data from the 20 CSV files 


### Data cleaning and preprocessing

In [3]:
# Drop unwanted columns
PM10_df.drop(['Parameter Code', 
            'Sample Duration', 
            'Pollutant Standard', 
            'Event Type', 
            'Observation Count', 
            'Observation Percent',
            'Method Code', 
            'Method Name',  
            'Local Site Name', 
            'Address', 
            'Date of Last Change'],axis=1,inplace=True)

In [4]:
# Drop Mexico, Virgin Islands and Puerto Rico
PM10_df.drop(PM10_df[(PM10_df['State Code']==80)|(PM10_df['State Code']==78)|(PM10_df['State Code']==72)].index,inplace=True)

In [5]:
# Aggregating so we have 1 measurement per station per day per parameter
PM10_df = PM10_df.groupby(['State Code', 
                       'State Name', 
                       'County Code', 
                       'County Name', 
                       'City Name', 
                       'Site Num', 
                       'Datum', 
                       'Latitude', 
                       'Longitude', 
                       'Date Local', 
                       'Parameter Name', 
                       'Units of Measure']).agg({'Arithmetic Mean':'mean', '1st Max Value':'max','1st Max Hour':'max','CBSA Name':'first','AQI':'mean'}).reset_index()

In [6]:
# Create unique identifier for each measurement
PM10_df['measurement_id'] = PM10_df['State Code'].apply(str) + '_' + PM10_df['County Code'].apply(str) + '_' + PM10_df['Site Num'].apply(str) + '_' + PM10_df['Date Local'].str.replace('-','')

In [7]:
# checking for the parameters included in the dataframe
PM10_df['Parameter Name'].unique()

array(['PM10 Total 0-10um STP'], dtype=object)

In [8]:
# Renaming column to match the value measured and droping unnecessary columns
PM10_df.rename(columns={'Arithmetic Mean':'PM10 Tot_0-10um STP(μg/m3)','1st Max Value':'PM10 Tot_0-10um STP(μg/m3) Max Value', '1st Max Hour':'PM10 Tot_0-10um STP(μg/m3) Max Hour'},inplace=True)
PM10_df.drop(['Parameter Name', 'Units of Measure'],axis=1,inplace=True)

In [9]:
# Convert date column to datetime
PM10_df['Date Local'] = pd.to_datetime(PM10_df['Date Local'])

In [10]:
PM10_df.head()

,State Code,State Name,County Code,County Name,City Name,Site Num,Datum,Latitude,Longitude,Date Local,PM10 Tot_0-10um STP(μg/m3),PM10 Tot_0-10um STP(μg/m3) Max Value,PM10 Tot_0-10um STP(μg/m3) Max Hour,CBSA Name,AQI,measurement_id
0,1,Alabama,53,Escambia,Brewton,2,NAD83,31.106567,-87.071081,2004-01-04,14.0,14.0,0,None,13.0,1_53_2_20040104
1,1,Alabama,53,Escambia,Brewton,2,NAD83,31.106567,-87.071081,2004-01-10,17.0,17.0,0,None,16.0,1_53_2_20040110
2,1,Alabama,53,Escambia,Brewton,2,NAD83,31.106567,-87.071081,2004-01-16,21.0,21.0,0,None,19.0,1_53_2_20040116
3,1,Alabama,53,Escambia,Brewton,2,NAD83,31.106567,-87.071081,2004-01-22,19.0,19.0,0,None,18.0,1_53_2_20040122
4,1,Alabama,53,Escambia,Brewton,2,NAD83,31.106567,-87.071081,2004-01-28,12.0,12.0,0,None,11.0,1_53_2_20040128


In [11]:
#Saving final df to csv
#PM10_df.to_csv(r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Cleaning\all_PM10_clean.csv",index=False)
PM10_df.to_csv(r"C:\Users\stlva\Google Drive\Big blue data academy\main_branch\concatenated data\all_PM10_clean.csv",index=False)